In [1]:
import numpy as np

In [5]:
# Creating the helper functions
# sigmoid function:

def sigmoid(x):
    s = 1/ (1 + np.exp(-x))
    return s

In [3]:
# Initialising parameters
# Creating vector with zeroes and initialising w and b param

def initialise(v):
    """
    Create a vector of np.shape() = (v, 1) and initialise b to 0
    """
    w = np.zeros((v, 1))
    b = 0 
    return w, b

In [10]:
# Forward and Backward Propogation

def propogation(w, b, X, Y):
    """
    Implement cost function and its gradient for the propogation
    """
    m = X.shape[1] # m is used to average out the cost function and so it is the total number of columns in X
    
    # Forward Propogation
    A = sigmoid(np.dot(w.T, X) + b)
    cost = (-1 / m) * np.sum(Y * np.log(A) + (1 - Y)* np.log(1 - A)) 
    
    # Backward Propogation
    dw = None
    db = None
    
    grads = {"dw": dw, "db": db}
    
    return grads, cost
    